In [7]:
import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [8]:
load_dotenv(override = True)

True

In [9]:
api_key = os.getenv('OPENAI_API_KEY')

MODEL = 'gpt-4o-mini'
openai = OpenAI()

In [10]:
class Website:
    """
    A utility class to represent a Website that we have scraped with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url)
        html = response.content
        soup = BeautifulSoup(html, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [26]:
wb = Website("https://rokomari.com/")
print(f"Title: {wb.title}")
print(f"Total Links: {len(wb.links)}")
print(wb.links[:10])

Title: Rokomari.com - Online Book, Electronics & Super Shop in Bangladesh
Total Links: 729
['/my-section/orders', '/offer', 'https://docs.google.com/forms/d/e/1FAIpQLScESDTv8m3eGL-E0Wwpq-8UbmhnQMYcWtteydzugmPO7QjbEw/viewform', 'https://affiliate.rokomari.io/', '/book-donation', '/book', '/login', '/cart', '/book?ref=nm', '/electronics?ref=nm']


In [16]:
# wb = Website("https://sayed-delowar.netlify.app/")
# print(f"Title: {wb.title}")
# print(f"Total Links: {len(wb.links)}")
# print(wb.links[:10])

In [17]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [18]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}



In [19]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [27]:
# print(get_links_user_prompt(wb))

In [31]:
def get_useful_links_with_openai(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model = MODEL,
        messages = [
            {"role":"system", "content":link_system_prompt},
            {"role":"user", "content":get_links_user_prompt(website)}
        ],
        response_format={"type":"json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)
        

In [32]:
# Get usefull links to make brochure using our agent
links_for_brochure = get_useful_links_with_openai("https://rokomari.com/")

In [33]:
links_for_brochure

{'links': [{'type': 'about page', 'url': 'https://www.rokomari.com/aboutus'},
  {'type': 'careers page', 'url': 'https://www.rokomari.com/retailer/signup'},
  {'type': 'company page', 'url': 'https://www.rokomari.com'},
  {'type': 'contact page', 'url': 'https://www.rokomari.com/contactus'},
  {'type': 'FAQ page', 'url': 'https://www.rokomari.com/faq'},
  {'type': 'help page', 'url': 'https://www.rokomari.com/help'},
  {'type': 'sitemap page', 'url': 'https://www.rokomari.com/sitemap'},
  {'type': 'reviews page', 'url': 'https://www.rokomari.com/reviews?ref=f_r'},
  {'type': 'terms page', 'url': 'https://www.rokomari.com/terms'},
  {'type': 'policy page', 'url': 'https://www.rokomari.com/policy'},
  {'type': 'social media', 'url': 'https://www.facebook.com/rokomari2.0'},
  {'type': 'instagram', 'url': 'https://www.instagram.com/rokomari/'},
  {'type': 'linkedin', 'url': 'https://www.linkedin.com/company/10239632'},
  {'type': 'youtube channel',
   'url': 'https://www.youtube.com/channe

In [28]:
# Get usefull links to make brochure using our agent
rokomari = Website("https://rokomari.com/")


## Making Brochure

In [38]:
def get_all_details(url):
    details = "Landing page:\n"
    details += Website(url).get_contents()

    # call the agent we made for getting useful links
    brochure_links = get_useful_links_with_openai(url)

    for link in brochure_links["links"]:
        details += f"""\n\n{link["type"]}\n"""
        details += Website(link["url"]).get_contents()
    
    return details

In [39]:
# Check all details are found or not
get_all_details("https://rokomari.com/")

'Landing page:\nWebpage Title:\nRokomari.com - Online Book, Electronics & Super Shop in Bangladesh\nWebpage Contents:\nরকমারিতে স্বাগতম!\nঅর্ডার ট্র্যাক করুন\nআজকের ডিল!\nরকমারি উদ্যোক্তা\nঘরে বসে আয় করুন\nবই ডোনেশন\nSign In\nবই\nইলেক্ট্রনিক্স\nসুপার স্টোর\nকিডস জোন\nবেস্টসেলার অ্যাওয়ার্ড, ২৪\nপ্রাতিষ্ঠানিক অর্ডার\nরকমারি কুইজ\nJust for you\nলেখক\nহুমায়ূন আহমেদ\nমুহম্মদ জাফর ইকবাল\nসমরেশ মজুমদার\nরবীন্দ্রনাথ ঠাকুর\nসুনীল গঙ্গোপাধ্যায়\nআনিসুল হক\nশীর্ষেন্দু মুখোপাধ্যায়\nসত্যজিৎ রায়\nআহমদ ছফা\nবিভূতিভূষণ বন্দ্যোপাধ্যায়\nসৈয়দ শামসুল হক\nসাদাত হোসাইন\nতামিম শাহরিয়ার সুবিন\nকাজী নজরুল ইসলাম\nহুমায়ুন আজাদ\nজহির রায়হান\nড্যান ব্রাউন\nচেতন ভগত\nরকিব হাসান\nসৈয়দ মুজতবা আলী\nমানিক বন্দ্যোপাধ্যায়\nবঙ্কিমচন্দ্র চট্টোপাধ্যায়\nআখতারুজ্জামান ইলিয়াস\nইমদাদুল হক মিলন\nসেলিনা হোসেন\nস্টিফেন কিং\nজে. কে. রাওলিং\nস্কাজী আনোয়ার হোসেন\nসুমন্ত আসলাম\nপাওলো কোয়েলহো\nশওকত ওসমান\nShow more\nবিষয়\nঅতিরিক্ত ছাড়\nবইমেলা ২০২৫\nইসলামি বই\nউপন্যাস\nকম্পিউটার, ফ্রিল্যান্সিং ও প্রোগ্রামিং\nজীবনী, স্মৃতিচারণ ও সাক্

In [40]:
# System prompt for creating brochure
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a concise, engaging brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information. Keep the tone professional yet inviting."

In [41]:
# Generate user prompt to create brochure
def get_brochure_user_prompt(company_name, url):
    user_prompt = f" You are looking at the company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevent pages. Use this information to build a consice, engaging brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:10000]

    return user_prompt

In [43]:
# Create gpt agent for creating the brochure
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model = MODEL,
        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
        ],
        stream = True
    )
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```", "").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [52]:
# Get the brochure now
stream_brochure("Rokomari", "https://www.rokomari.com/")

# Welcome to Rokomari.com

### Your Ultimate Online Shopping Destination in Bangladesh

---

## 📖 Explore a World of Books
At Rokomari, we pride ourselves on having a vast collection of over **300,000 books** spanning nearly **3,000 categories**. Whether you are a fan of fiction, non-fiction, academic literature, or creative excellence, we have titles by renowned authors like **Humayun Ahmed, Muhammad Zafar Iqbal, J.K. Rowling**, and many more!

### 📦 Beyond Books
Rokomari is not just about books; we are your one-stop online shop for **electronics, kids’ items, kitchen appliances**, and much more! We cater to over **500 categories** to ensure that all your shopping needs are met at one convenient location.

---

## 🌟 Why Choose Rokomari?
- **User-Friendly Experience**: Our website makes finding and buying your favorite items a breeze. 
- **Fast and Reliable Delivery**: Thanks to our strong distribution network, we ensure timely delivery across **Bangladesh and beyond**.
- **Diverse Customer Base**: From students looking for textbooks to families shopping for groceries, we serve a wide range of customers.

---

## 🎉 Company Culture
At Rokomari, we foster an inclusive and dynamic workplace where **innovation meets tradition**. Our team is passionate about promoting literacy and accessibility through innovative solutions in e-commerce. We believe in empowering each teammate to make an impact and deliver unparalleled service to our customers.

---

## 🚀 Career Opportunities
As we grow, we invite **talented individuals** to join our team. If you are looking for a vibrant workplace where creativity and initiative are valued, then Rokomari is the place for you! Explore career opportunities across various departments and contribute to redefining the shopping experience in Bangladesh.

### Join Us:
- **Sales & Marketing**
- **Customer Service**
- **IT & Tech Development**
- **Logistics and Distribution**

---

## 📞 Get in Touch
For inquiries or to see how you can partner with us, reach out! 

- **Customer Support**: [Email Us](mailto:support@rokomari.com) 
- **Corporate Sales**: [Corporate Inquiries](mailto:corporate@rokomari.com) 

### Visit Us:
**Location**: 2/1/E, Eden Center, Arambag, Motijheel, Dhaka-1000

---

## 🌐 Discover the Rokomari Experience
Dive into a seamless online shopping experience today! Visit us at [rokomari.com](https://www.rokomari.com) and start exploring the possibilities.

---

*Rokomari - Revolutionizing online shopping in Bangladesh, one book at a time.*

## With Gradio UI

In [55]:
# Gradio

force_dark_mode = """
function refresh() {
    const url = new URL(window.location);
    if (url.searchParams.get('__theme') !== 'dark') {
        url.searchParams.set('__theme', 'dark');
        window.location.href = url.href;
    }
}
"""

# gr.Interface(fn=shout, inputs="textbox", outputs="textbox", flagging_mode="never", js=force_dark_mode).launch(share=True)

view = gr.Interface(
    fn=stream_brochure,
    inputs=[
        gr.Textbox(label="Company Name"),
        gr.Textbox(label="Landing page URL including http:// or https://"),
    ],
    outputs=[gr.Markdown(label="Brochure")],
    flagging_mode="never",
    js=force_dark_mode
)
view.launch()

* Running on local URL:  http://127.0.0.1:7868

To create a public link, set `share=True` in `launch()`.


# Building Technology & Ideas Ltd. (BTI) - Your Trusted Real Estate Partner in Bangladesh

---

## **About Us**

Welcome to BTI! Since 1984, Building Technology & Ideas Ltd. has been a pioneer in the real estate sector of Bangladesh. Our commitment to excellence has allowed us to deliver quality residential and commercial properties with the utmost professionalism. With over 400 completed projects and more than 7,000 satisfied customers, we stand as a testament to reliability, innovation, and customer-centricity.

---

## **Our Offerings**

### **Diverse Property Categories:**
- **Classic Collection:** Timeless architecture that combines functionality with elegance.
- **Luxury Collection:** Premium residential spaces designed for the upscale market.
- **Wellness Communities:** Thoughtfully designed environments promoting health and community living.
- **Commercial Spaces:** Strategically located properties tailored for businesses.

### **Special Services:**
- **bti Building Products:** Innovative construction materials that enhance quality while being environmentally friendly.
- **Get Smart by bti:** Home automation solutions to elevate your living experience.
- **Before & After Sales Services:** Exceptional support from property purchase to efficient maintenance management.

---

## **Our Commitment to Excellence**

- **Trustworthy and Reliable:** We deliver what we promise, often ahead of schedule.
- **High-Grade Construction:** We adhere to the highest building codes, ensuring safety and durability.
- **Modern Design Philosophy:** Our design mantra is simplicity and practicality, ensuring each space meets the needs of our customers.
- **Customer First Approach:** We provide personalized experiences through dedicated teams aimed at fulfilling your unique lifestyle choices.

---

## **Company Culture**

At BTI, our culture thrives on innovation, integrity, and a strong commitment to our customers. We believe in going the extra mile to ensure customer satisfaction, from guiding you through the property selection process to providing continuous support post-purchase. Our in-house team of architects, planners, and engineers work collaboratively to create spaces that transform lives.

---

## **Careers at BTI**

Are you looking for a fulfilling career with one of Bangladesh's leading real estate firms? We are always on the lookout for talented professionals who share our passion for excellence and customer satisfaction. Join us at BTI to build a remarkable career in real estate, architecture, and construction.

---

## **Connect with Us**

Take the first step towards realizing your property dreams or explore potential partnerships. Contact us today!

**Phone:** +8809613 191919 | +8801313 401405  
**Email:** [contact@bti.com.bd](mailto:contact@bti.com.bd)  
**Visit Us:** bti Celebration Point, Gulshan-2, Dhaka-1212

---

## **Stay Updated**

Subscribe to our newsletter for the latest updates, project launches, and exclusive offers!

**Join us as we continue to build dreams and create spaces that people will call home!**

--- 

*Experience the difference with BTI - where your vision meets our expertise.* 